# PA5 - Naive Bayes

## Step 1 - Train Dataset

For this step, we will create a Naive Bayes classifier for the "train" dataset, provided below. 

In [10]:
table = [
    ["weekday", "spring", "none", "none", "on time"],
    ["weekday", "winter", "none", "slight", "on time"],
    ["weekday", "winter", "none", "slight", "on time"],
    ["weekday", "winter", "high", "heavy", "late"], 
    ["saturday", "summer", "normal", "none", "on time"],
    ["weekday", "autumn", "normal", "none", "very late"],
    ["holiday", "summer", "high", "slight", "on time"],
    ["sunday", "summer", "normal", "none", "on time"],
    ["weekday", "winter", "high", "heavy", "very late"],
    ["weekday", "summer", "none", "slight", "on time"],
    ["saturday", "spring", "high", "heavy", "cancelled"],
    ["weekday", "summer", "high", "slight", "on time"],
    ["saturday", "winter", "normal", "none", "late"],
    ["weekday", "summer", "high", "none", "on time"],
    ["weekday", "winter", "normal", "heavy", "very late"],
    ["saturday", "autumn", "high", "slight", "on time"],
    ["weekday", "autumn", "none", "heavy", "on time"],
    ["holiday", "spring", "normal", "slight", "on time"],
    ["weekday", "spring", "normal", "none", "on time"],
    ["weekday", "spring", "normal", "slight", "on time"]
]

To create this classifier, we first must calculate prior probabilities for each class label.
We will do this with the following helper function:

* `calculate_priors()`
    * **Params**:
        * `data` - The dataset to calculate prior probabilities for
        * `index` - The index of the classifier value in the dataset
        * `classes` - An array of the classes present in the dataset
    * **Returns**:
        * An array of prior probability values, ordered according to the `classes` param.

In [11]:
def calculate_priors(data, index, classes):
    counts = {}
    total = 0
    for label in classes:
        counts[label] = 0
    for instance in data:
        counts[instance[index]] += 1
        total += 1
    probabilities = []
    for label in classes:
        probabilities.append(counts[label] / total)
    return probabilities

To check these values, we will refer to **Figure 3.2** from the Bramer textbook, which claims that the Prior Probabilities for "on time", "late" "very late", and "cancelled" should be 0.70, 0.10, 0.15, and 0.05, respectively.

We will run our `calculatePriors()` function on these labels and display the values, which should match the given probabilities.

In [12]:
priors = calculate_priors(table, 4, ["on time", "late", "very late", "cancelled"])
print(priors)

[0.7, 0.1, 0.15, 0.05]


As we can see, the values returned from `calculate_priors()` are the same as given in Bramer. Next, we will calculate the posterior probabilities for a given class label. Again, we will create a helper function `calculate_posteriors()` to find these values:

* `calculate_posteriors()`
    * **Params**:
        * `data` - The dataset to calculate probabilities for
        * `attributeIndex` - The index of the attribute to calculate conditional probability for
        * `attribute` - The value of the index to calculate conditional probability for
        * `classIndex` - The index of the class label
        * `classLabels` - All classifier labels to calculate conditional probabilities for
    * **Returns**:
        * A list of posterior probabilities for the given attribute over each class label, ordered with respect to `classLabels()`

In [13]:
def calculate_posteriors(data, attributeIndex, attribute, classIndex, classLabels):
    conditionalCounts = {}
    counts = {}
    for label in classLabels:
        counts[label] = 0
        conditionalCounts[label] = 0
    for instance in data:
        counts[instance[classIndex]] += 1
        if instance[attributeIndex] == attribute:
            conditionalCounts[instance[classIndex]] += 1
    probabilities = []
    for label in classLabels:
        if counts[label] == 0:
            probabilities.append(0)
        else:
            probabilities.append(conditionalCounts[label] / counts[label])
    return probabilities

Once again, we will check our function output against the values provided by Bramer. For the attribute (day = "weekday"), we expect the posterior probabilities for class = "on time", "late", "very late", and "cancelled" to be 0.64, 0.5, 1, and 0, respectively.

In [14]:
posteriors = calculate_posteriors(table, 0, "weekday", 4, ["on time", "late", "very late", "cancelled"])
print(posteriors)

[0.6428571428571429, 0.5, 1.0, 0.0]


Again, our values match up, although Bramer's values round off to 2 decimal places while ours sometimes have more bits of precision.

Finally, we can use this to create a Naive Bayes classifier function.

* `naive_bayes_classify()`
    * **Params**:
        * `train` - The data to use as training data
        * `classIndex` - The index of the class label
        * `classLabels` - A list of all possible class labels
        * `test` - The unseen data to classify
    * **Returns**:
        * A classification for the `test` instance

In [15]:
def naive_bayes_classify(train, classIndex, classLabels, test):
    classProbabilities = []
    for val in classLabels:
        classProbabilities.append(0)
    priors = calculate_priors(train, classIndex, classLabels)
    for i in range(len(classProbabilities)):
        classProbabilities[i] += priors[i]
    for i in range(len(test)):
        if i == classIndex:
            continue
        else:
            attribute = test[i]
            posteriors = calculate_posteriors(train, i, attribute, classIndex, classLabels)
            for i in range(len(posteriors)):
                classProbabilities[i] *= posteriors[i]
    maxP = 0
    index = 0
    for i in range(len(classProbabilities)):
        if classProbabilities[i] > maxP:
            maxP = classProbabilities[i]
            index = i
    return classLabels[index]
            

To test that our classifier is functioning as intended, we will test it on the trains dataset using the unseen value ("weekday", "winter", "high", "heavy", "???"). Accordign to Bramer, this should be classified as "very late".

In [16]:
label = naive_bayes_classify(table, 4, ["on time", "late", "very late", "cancelled"], ["weekday", "winter", "high", "heavy", "???"])
print(label)

very late


And, as shown, the `naive_bayes_classify()` function does indeed classify the unseen data correctly. Therefore, we now have a functioning method for using Naive Bayes classification accross a given dataset.

## Step 2 - MPG predictor

For this step, we will use our Naive Bayes methods on the auo-data dataset. First, we will import the data into an array titled `auto_data`. To generate this data, we will reuse the functions `read_data()`, `create_dataset()`, and `resolve_missing()` from PA4.

In [55]:
def read_data(filename):
    f = open(filename, 'r')
    text = f.read()
    f.close()
    return text

def numerify_instance(instance):
    new = []
    for attribute in instance:
        try:
            new.append(float(attribute))
        except:
            new.append(attribute)
    return new

def create_dataset(data):
    data_r = data.splitlines()
    dataset_r = []
    for line in data_r:
        instance = line.split(',')
        dataset_r.append(instance)
    dataset = []
    for instance in dataset_r:
        newInstance = numerify_instance(instance)
        dataset.append(newInstance)
    return dataset

def resolve_missing_values(data):
    for i in range(10):
        if i != 8:
            sum_i = 0
            count_i = 0
            for instance in data:
                if instance[i] != "NA":
                    try:
                        sum_i += instance[i]
                        count_i += 1
                    except:
                        print(instance[i])
            if count_i == 0:
                continue
            mean = sum_i / count_i
            for instance in data:
                if instance[i] == "NA":
                    instance[i] = mean

Then, we will use these functions to populate `auto_data`.

In [56]:
auto_data_r = create_dataset(read_data("auto-data.txt"))
resolve_missing_values(auto_data_r)

This step only cares about the cyliders, weight, and model year attributes, as well as mpg as a classifier. So, to clean the dataset, we will first go through and restrict it to only these values.

In [57]:
def clean_auto_data(data):
    cleaned_auto_data = []
    for instance in data:
        cleaned_instance = [instance[1], instance[4], instance[6], instance[0]]
        cleaned_auto_data.append(cleaned_instance)
    return cleaned_auto_data

auto_data_c = clean_auto_data(auto_data_r)

Then, we will go through and discretize mpg based on the DOE classification ranking, as well as weight based on the NHTSA vehicle sizes classification. Both tables are given below for reference.

| Rating | MPG   |
|--------|-----  |
|   10   | ≥ 45  |
|   9    | 37-44 |
|   8    | 31-36 |
|   7    | 27-30 |
|   6    | 24-26 |
|   5    | 20-23 |
|   4    | 17-19 |
|   3    | 15-16 |
|   2    |   14  |
|   1    | ≤ 13  |

| Ranking |  Weight   |
|---------|-----------|
|    5    | ≥ 3500    |
|    4    | 3000-3499 |
|    3    | 2500-2999 |
|    2    | 2000-2499 |
|    1    | ≤ 1999    |

In [58]:
def mpg_to_DOE(mpg):
    if mpg >= 45:
        y = 10
    elif mpg >= 37:
        y = 9
    elif mpg >= 31:
        y = 8
    elif mpg >= 27:
        y = 7
    elif mpg >= 24:
        y = 6
    elif mpg >= 20:
        y = 5
    elif mpg >= 17:
        y = 4
    elif mpg >= 15:
        y = 3
    elif mpg >= 14:
        y = 2
    else:
        y = 1
    return y

def weight_to_NHTSA(weight):
    if weight >= 3500:
        return 5
    elif weight >= 3000:
        return 4
    elif weight >= 2500:
        return 3
    elif weight >= 2000:
        return 2
    else:
        return 1

    
def discretize_auto_data(data):
    discrete_data = []
    for instance in data:
        discrete = [instance[0], weight_to_NHTSA(instance[1]), instance[2], mpg_to_DOE(instance[3])]
        discrete_data.append(discrete)
    return discrete_data

auto_data = discretize_auto_data(auto_data_c)

We will now test our classifier by repeating steps 2-5 from PA4.

### Random Instances

First, we will test our classifier on a subset of 5 random instances from the dataset. To do so, we must first generate 5 random instances to test. We will reuse `generate_random_instances()` from PA4.

In [59]:
from random import randint
import copy

def generate_random_instances(data, n):
    data_c = copy.deepcopy(data)
    test_instances = []
    for i in range(n):
        index = randint(0, len(data_c)-1)
        instance = data_c.pop(index)
        test_instances.append(instance)
    return test_instances

random_test = generate_random_instances(auto_data, 5)

Then, we will use our `naive_bayes_classify()` function to classify each test instance.

In [60]:
def generate_predictions(train, test):
    predictions = []
    for instance in test:
        predictions.append(naive_bayes_classify(train, 3, [1, 2, 3, 4, 5, 6, 7, 8, 9, 10], instance))
    return predictions

def print_output(test, predictions):
    for x in range(len(test)):
        print("instance: ", test[x][0], ", ", test[x][1], ", ", test[x][2], sep="")
        print("predicted: ", predictions[x], ", actual: ", test[x][3], sep="")
        print()

Now that we can generate a list of predictions, we will run the classifier and output the results.

In [61]:
predictions = generate_predictions(auto_data, random_test)
print_output(random_test, predictions)


instance: 4.0, 3, 72.0
predicted: 5, actual: 4

instance: 4.0, 2, 72.0
predicted: 7, actual: 7

instance: 4.0, 2, 79.0
predicted: 7, actual: 8

instance: 4.0, 1, 77.0
predicted: 8, actual: 7

instance: 8.0, 5, 76.0
predicted: 1, actual: 1



### Test / Train sets

For this test, we will use stratified cross validation to create a 2:1 train/test set. To generate the train and test sets, we will once again reuse functions from PA4: `create_random_subsample()`, `compute_accuracy()`, and `compute_error()`.

In [62]:
from random import shuffle

def create_random_subsample(data, size):
    cutoff = int(len(data) * size)
    data_c = copy.deepcopy(data)
    shuffle(data_c)
    train = data_c[:cutoff]
    test = data_c[cutoff + 1:]
    return test, train

def compute_accuracy(predictions, actual):
    correct = 0
    for i in range(len(predictions)):
        if predictions[i] == actual[i]:
            correct += 1
    accuracy = correct / len(predictions)
    return accuracy

def compute_error(predictions, actual):
    incorrect = 0
    for i in range(len(predictions)):
        if predictions[i] != actual[i]:
            incorrect += 1
    error = incorrect / len(predictions)
    return error

Then we can use these functions to generate predictions and print the accuracy and the error rate.

In [63]:
test, train = create_random_subsample(auto_data, 2/3)
predictions = generate_predictions(train, test)
accuracy = compute_accuracy(predictions, [x[3] for x in test])
error = compute_error(predictions, [x[3] for x in test])

print("Accuracy: ", accuracy)
print("Error Rate: ", error)

Accuracy:  0.38461538461538464
Error Rate:  0.6153846153846154


### Stratified Cross Validation

Finally, we will use Stratified 10-Fold Cross Validation to generate random subsamples of test / train data to run our classifier on. 

We will borrow the function `create_cross_fold()` from PA4 for doing the subsampling.

In [64]:
def create_cross_fold(data, n):
    data_r = copy.deepcopy(data)
    shuffle(data_r)
    size = int(len(data) * 1/n) 
    start = 0
    end = size
    folds = []
    for i in range(n-1):
        folds.append(data[start:end])
        start = end + 1
        end += size + 1
    folds.append(data[start:])
    return folds

We now need to run the classifier 10 times, using each subsequent fold as test data and the other 9 folds as training data. For each fold, we will compute an accuracy and error rate, and then display the average at the end.

In [65]:
def print_cross_fold_output(data, n):
    sum_accuracy = 0
    sum_error = 0
    folds = create_cross_fold(data, n)
    for i in range(n):
        test = folds[i]
        train = []
        for x in range(n):
            if x == i:
                continue
            train += folds[x]
        predictions = generate_predictions(train, test)
        sum_accuracy += compute_accuracy(predictions, [x[3] for x in test])
        sum_error += compute_error(predictions, [x[3] for x in test])
    accuracy = sum_accuracy / n
    error = sum_error / n
    print("Accuracy:", accuracy)
    print("Error Rate:", error)
    
print_cross_fold_output(auto_data, 10)

Accuracy: 0.3471923536439666
Error Rate: 0.6528076463560335


## Step 3 - Gaussian Distribution

For this step, we will repeat step 2 treating weight as a continuous attribute. To do this, we will utilize the Gaussian distribution function to calculate its conditional probability.

First, we will implement the `gaussian()` functions, referenced from [U4 - Supervised Learning](https://github.com/GonzagaCPSC310/U4-Supervised-Learning/blob/master/D%20Naive%20Bayes.ipynb), defined by:



In [66]:
import math
def gaussian(x, mean, sdev):
  first, second = 0, 0
  if sdev > 0:
      first = 1 / (math.sqrt(2 * math.pi) * sdev)
      second = math.e ** (-((x - mean) ** 2) / (2 * (sdev ** 2)))
  return first * second

We can now redefine our classifier, using the Gaussian function for weight. 

We will also need helper functions in order to calculate the mean and standard deviation of a given attribute for a given class label. 


* `class_std_dev_attribute()`
    * **Params**:
        * `data` - The dataset to calculate standard deviation over
        * `index` - The index of the attribute to calculate standard deviation for
        * `classLabel` - The class label to calculate the std dev for
        * `classIndex` - the index of the classifier attribute in the data
    * **Returns**:
        * The standard deviation of the given attribute in the dataset over all instances where the class label is `classLabel`
* `class_mean_attribute()`
    * **Params**:
        * `data` - The dataset to calculate mean over
        * `index` - The index of the attribute to calculate mean for
        * `classLabel` - The class label to calculate the mean for
        * `classIndex` - the index of the classifier attribute in the data
    * **Returns**:
        * The mean of the given attriute over all instances where the class label is `classLabel`
* `naive_bayes_classify_gaussian()`
    * **Params**:
        * `train` - The dataset to use for the training set
        * `classIndex` - The index of the classifier attribute within the data
        * `classLabels` - A list of all possible class labels
        * `gaussianIndices` - A list of indices to treat as continuous variables and use gaussian distributions on
        * `test` - The unseen test data to predict a class for
    * **Returns**:
        * A predicted classification using Naive Bayes, calculated using posterior probabilities for categorical variables and gaussian distributions for continuous variables.

In [67]:
def class_mean_attribute(data, index, classLabel, classIndex):
    sum = 0
    count = 0
    for instance in data:
        if instance[classIndex] == classLabel:
            sum += instance[classIndex]
            count += 1
    if count == 0:
        return 0
    else:
        return sum / count

def class_std_dev_attribute(data, index, classLabel, classIndex):
    sum = 0
    count = 0
    mean = class_mean_attribute(data, index, classLabel, classIndex)
    for instance in data:
        if instance[classIndex] == classLabel:
            sum += (instance[index] - mean)**2
            count += 1
    if count == 0:
        return 0
    else:
        return ((sum / count)**(1/2))


def naive_bayes_classify_gaussian(train, classIndex, classLabels, gaussianIndices, test):
    classProbabilities = []
    for val in classLabels:
        classProbabilities.append(0)
    priors = calculate_priors(train, classIndex, classLabels)
    for i in range(len(classProbabilities)):
        classProbabilities[i] += priors[i]
    for i in range(len(test)):
        if i == classIndex:
            continue
        elif i in gaussianIndices:
            gaussians = []
            for label in classLabels:
                mean = class_mean_attribute(train, i, label, classIndex)
                std_dev = class_std_dev_attribute(train, i, label, classIndex)
                gaussians.append(gaussian(test[i], mean, std_dev))
            for i in range(len(gaussians)):
                classProbabilities[i] *= gaussians[i]
        else:
            attribute = test[i]
            posteriors = calculate_posteriors(train, i, attribute, classIndex, classLabels)
            for i in range(len(posteriors)):
                classProbabilities[i] *= posteriors[i]
    maxP = 0
    index = 0
    for i in range(len(classProbabilities)):
        if classProbabilities[i] > maxP:
            maxP = classProbabilities[i]
            index = i
    return classLabels[index]
            

Now, we need to recreate the dataset, since weight is no longer a categorical variable. We will copy the previous `discretize_auto_data()` function, leaving weight as is instead of converting to NHTSA rankings.

In [68]:
def discretize_auto_data_gaussian(data):
    discrete_data = []
    for instance in data:
        discrete = [instance[0], instance[1], instance[2], mpg_to_DOE(instance[3])]
        discrete_data.append(discrete)
    return discrete_data

auto_data_g = discretize_auto_data_gaussian(auto_data_c)

We can now recreate the phases of Step 2, using this new gaussian classifier.

### Random Instances

First, we will test the Gaussian classifier against 5 random instances. We must modify our `generate_predictions()` function in order to use the gaussian classifier, which we will rename `generate_gaussian_predictions()`.

In [69]:
def generate_gaussian_predictions(train, test):
    predictions = []
    for instance in test:
        predictions.append(naive_bayes_classify_gaussian(train, 3, [1, 2, 3, 4, 5, 6, 7, 8, 9, 10], [1], instance))
    return predictions

Now, we will use the same random generation function and prediction output function as before to test our gaussian prediction on a random subsample of the data.

In [70]:
random_test_g = generate_random_instances(auto_data_g, 5)
predictions_g = generate_gaussian_predictions(auto_data_g, random_test_g)
print_output(random_test_g, predictions_g)

instance: 4.0, 2694.0, 75.0
predicted: 6, actual: 5

instance: 4.0, 2155.0, 76.0
predicted: 7, actual: 7

instance: 6.0, 2904.0, 73.0
predicted: 4, actual: 5

instance: 4.0, 2130.0, 70.0
predicted: 6, actual: 7

instance: 8.0, 4440.0, 75.0
predicted: 3, actual: 3



### Train/Test Sets: Random Sub-Sampling

Second, we will once again generate a random 2:1 split of train / test data and perform our prediction using the gaussian classifier. This will follow the exact same format as the original classifier, using the new datasets.

In [81]:
test, train = create_random_subsample(auto_data_g, 2/3)
predictions = generate_gaussian_predictions(train, test)
accuracy = compute_accuracy(predictions, [x[3] for x in test])
error = compute_error(predictions, [x[3] for x in test])

print("Accuracy: ", accuracy)
print("Error Rate: ", error)

Accuracy:  0.3076923076923077
Error Rate:  0.6923076923076923


### Train/Test Sets: Stratified Cross Validation

Finally, we will recreate our 10-fold cross validation testing using the gaussian classifier. We will reuse the `print_cross_fold_output()` function, rewritten to use our gaussian classifier.

In [83]:
def print_cross_fold_output_gaussian(data, n):
    sum_accuracy = 0
    sum_error = 0
    folds = create_cross_fold(data, n)
    for i in range(n):
        test = folds[i]
        train = []
        for x in range(n):
            if x == i:
                continue
            train += folds[x]
        predictions = generate_gaussian_predictions(train, test)
        sum_accuracy += compute_accuracy(predictions, [x[3] for x in test])
        sum_error += compute_error(predictions, [x[3] for x in test])
    accuracy = sum_accuracy / n
    error = sum_error / n
    print("Accuracy:", accuracy)
    print("Error Rate:", error)


print_cross_fold_output_gaussian(auto_data_g, 10)

Accuracy: 0.3097968936678614
Error Rate: 0.6902031063321387


## Step 4 - Titanic Dataset

For this step, we will use both Naive Bayes and k-NN to predict the survival attribute from the titanic dataset, which we will now load. Note that this file has been cleaned from its origial form to only include the relevant data, without any headers or identifying information.

In [79]:
titanic_data = create_dataset(read_data("titanic_data.txt"))

Next, we define 2 functions similar to functions used previously: `generate_titanic_predictions()` and `print_titanic_cross_fold_output()`. These will allow us to use the Naive Bayes classifier on the titanic dataset. 

In [108]:
def generate_titanic_predictions(train, test):
    predictions = []
    for instance in test:
        predictions.append(naive_bayes_classify(train, 3, ["yes", "no"], instance))
    return predictions

def print_titanic_cross_fold_output(data, n):
    sum_accuracy = 0
    sum_error = 0
    folds = create_cross_fold(data, n)
    for i in range(n):
        test = folds[i]
        train = []
        for x in range(n):
            if x == i:
                continue
            train += folds[x]
        predictions = generate_titanic_predictions(train, test)
        sum_accuracy += compute_accuracy(predictions, [x[3] for x in test])
        sum_error += compute_error(predictions, [x[3] for x in test])
    accuracy = sum_accuracy / n
    error = sum_error / n
    print("Accuracy:", accuracy)
    print("Error Rate:", error)
    

We will also define a k-NN classifier for the dataset, borrowing from PA4 once more.

Since all of the attributes in the titanic dataset are categorical and non-numeric, we will convert them to normalized numeric values instead. Since Age, Sex, and Survived are binary, we will simply arbitrarily assign one value to 0 and one value to 1 for each attribute. For Class, however, there are 4 values: First, Second, Third, and Crew. Here, we make some assumptions about the data in the normalizing process. We can assume that passenger class likely had an impact on survival, since it would affect their living quarter placement as well as potentially their access to lifeboats; presumably, first class passengers would be given priority over crew. Therefore, we will order them with crew being 0 and first class being 1, to hopefully give an accurate normalization of the data.

The data will be normalized as follows:

| Class  | Normalized |
|--------|------------|
| crew   |     0      |
| third  |   0.333..  |
| second |   0.666..  |
| first  |     1      |

|  Age  | Normalized |
|-------|------------|
| child |     0      |
| adult |     1      |

|   Sex  | Normalized |
|--------|------------|
| female |      0     |
|  male  |      1     |

| Survived | Normalized |
|----------|------------|
|    no    |     0      |
|   yes    |     1      |

In [109]:
def normalize_titanic_data(data):
    normal_data = []
    class_n = {"crew":0, "third":(1/3), "second":(2/3), "first":1}
    age_n = {"child":0, "adult":1}
    sex_n = {"female":0, "male":1}
    survived_n = {"no":0, "yes":1}
    for instance in data:
        instance_n = []
        instance_n.append(class_n[instance[0]])
        instance_n.append(age_n[instance[1]])
        instance_n.append(sex_n[instance[2]])
        instance_n.append(survived_n[instance[3]])
        normal_data.append(instance_n)
    return normal_data

titanic_knn = normalize_titanic_data(titanic_data)

Now, we can write our k-NN classifier, copying heavily from PA4 but modifying the logic slightly to fit our needs better. We will need `calculate_distance()` to get the distance between two instances, `calculate_all_distances()` for getting a list of distances for the entire dataset, and `calculate_most_common_element()` for k-NN voting, as well as a modified `knn()` function as the actual classifier.

In [118]:
import operator

def calculate_distance(instance, base, class_index):
    sum_a = 0
    for i in range(len(instance)):
        if i == class_index:
            continue
        else:
            dif = instance[i] - base[i]
            sum_a += dif**2
    distance = sum_a**(1/2)
    return distance

def calculate_all_distances(data, base, class_index):
    distances = []
    for instance in data:
        distance = calculate_distance(instance, base, class_index)
        distances.append((instance[class_index], distance))
    return distances

def compute_most_common_element(data, index):
    freqs = {}
    for instance in data:
        if instance[index] in freqs:
            freqs[instance[index]] += 1
        else:
            freqs[instance[index]] = 1            
    # line referenced from https://stackoverflow.com/questions/268272/getting-key-with-maximum-value-in-dictionary
    mce = max(freqs.items(), key=operator.itemgetter(1))[0]
    return mce

def knn(data, instance, k, class_index):
    distances = calculate_all_distances(data, instance, class_index)
    sorted_distances = sorted(distances, key=operator.itemgetter(1))
    neighbors = []
    for i in range(k):
        neighbors.append(sorted_distances[i][0])
    return compute_most_common_element([[x] for x in neighbors], 0)

Finally, we need to define methods to generate multiple k-NN predictions, as well as perform a stratified cross-fold train/test split using k-NN.

In [113]:
def generate_knn_predictions(train, test):
    predictions = []
    for instance in test:
        predictions.append(knn(titanic_knn, instance, 5, 3))
    return predictions

def print_knn_cross_fold_output(data, n):
    sum_accuracy = 0
    sum_error = 0
    folds = create_cross_fold(data, n)
    for i in range(n):
        test = folds[i]
        train = []
        for x in range(n):
            if x == i:
                continue
            train += folds[x]
        predictions = generate_knn_predictions(train, test)
        sum_accuracy += compute_accuracy(predictions, [x[3] for x in test])
        sum_error += compute_error(predictions, [x[3] for x in test])
    accuracy = sum_accuracy / n
    error = sum_error / n
    print("Accuracy:", accuracy)
    print("Error Rate:", error)

Now that we have our classifiers defined, we can test their output. First, we will run a 10-fold cross validation on the Naive Bayes classifier:

In [114]:
print_titanic_cross_fold_output(titanic_data, 10)

Accuracy: 0.7774442538593482
Error Rate: 0.22255574614065182


And then a 10-fold cross validation on the k-NN classifier:

In [119]:
print_knn_cross_fold_output(titanic_knn, 10)

Accuracy: 0.7824614065180102
Error Rate: 0.2175385934819897


As we can see, both classifiers produce very similar accuracies, sitting around 78% accurate. To better display these results, we will also create two functions to print confusion matrices for both classifiers.

In [124]:
from tabulate import tabulate

def confusion_matrix_bayes(data, n):
    results = [["no", 0, 0], ["yes", 0, 0]]
    folds = create_cross_fold(data, n)
    for i in range(n):
        test = folds[i]
        train = []
        for x in range(n):
            if x == i:
                continue
            train += folds[x]
        predictions = generate_titanic_predictions(train, test)
        actual = [instance[3] for instance in test]
        for i in range(len(actual)):
            if actual[i] == "no":
                if predictions[i] == "no":
                    results[0][1] += 1
                else:
                    results[0][2] += 1
            else:
                if predictions[i] == "no":
                    results[1][1] += 1
                else:
                    results[1][2] += 1
    for row in results:
        row.append(sum(row[1:]))
        if row[0] == "no":
            row.append((row[1] / row[3]) * 100)
        else:
            row.append((row[2] / row[3]) * 100)
    header = ["Survived", "no", "yes", "Total", "Recognition (%)"]
    print("Naive Bayes (Stratefied", n, "-Fold Cross Validation Results):", sep="")
    print("===============================================================")
    print(tabulate(results, headers=header))
    
confusion_matrix_bayes(titanic_data, 10)

Naive Bayes (Stratefied10-Fold Cross Validation Results):
Survived      no    yes    Total    Recognition (%)
----------  ----  -----  -------  -----------------
no           346    362      708            48.8701
yes          126   1358     1484            91.5094


In [125]:
def confusion_matrix_knn(data, n):
    results = [["no", 0, 0], ["yes", 0, 0]]
    folds = create_cross_fold(data, n)
    for i in range(n):
        test = folds[i]
        train = []
        for x in range(n):
            if x == i:
                continue
            train += folds[x]
        predictions = generate_knn_predictions(train, test)
        actual = [instance[3] for instance in test]
        for i in range(len(actual)):
            if actual[i] == 0:
                if predictions[i] == 0:
                    results[0][1] += 1
                else:
                    results[0][2] += 1
            else:
                if predictions[i] == 0:
                    results[1][1] += 1
                else:
                    results[1][2] += 1
    for row in results:
        row.append(sum(row[1:]))
        if row[0] == "no":
            row.append((row[1] / row[3]) * 100)
        else:
            row.append((row[2] / row[3]) * 100)
    header = ["Survived", "no", "yes", "Total", "Recognition (%)"]
    print("k-NN Bayes (Stratefied", n, "-Fold Cross Validation Results):", sep="")
    print("===============================================================")
    print(tabulate(results, headers=header))
    
confusion_matrix_knn(titanic_knn, 10)

k-NN Bayes (Stratefied10-Fold Cross Validation Results):
Survived      no    yes    Total    Recognition (%)
----------  ----  -----  -------  -----------------
no           357    351      708            50.4237
yes          126   1358     1484            91.5094


## Step 5 - Baseline Classifiers

To compare the accuracy of the previously created Naive Bayes and k-NN classifiers on the Titanic dataset, we will create two "baseline" classifiers to use for reference. First, we will create a Random classifier, which will generate predictions randomly with guesses weighted based on the frequency of class labels in the dataset. Then, we will also create a Zero Rule classifier, which simply always guesses the most common class label in the dataset. The two functions for these classifiers are as follows:

* `random_classifier()`
    * **Params**:
        * `data` - The dataset to use for generating predictions
        * `n` - The number of random predictions to generate
        * `classIndex` - The index of the classification attribute in the data
    * **Returns**:
        * A list of `n` random classifications, weighted based on the frequency of class labels in the database (ie if 80% of the data is classified as Yes and 20% as No, the classifier will guess Yes about 80% of the time.)
* `zero_r_classifier()`
    * **Params**:
        * `data` - The dataset to use for generating predictions
        * `n` - The number of predictions to generate
        * `classIndex` - The index of the classification attribute in the data
    * **Returns**:
        * A list of `n` instances of the most common class label
        

In [128]:
def random_classifier(data, n, classIndex):
    classes = []
    for instance in data:
        classes.append(instance[classIndex])
    predictions = []
    for i in range(n):
        index = randint(0, len(classes))
        predictions.append(data[index][classIndex])
    return predictions


def zero_r_classifier(data, n, classIndex):
    mce = compute_most_common_element(data, classIndex)
    predictions = []
    for i in range(n):
        predictions.append(mce)
    return predictions

Now, knowing that our k-NN and Naive Bayes classifiers both have an accuracy of about 78%, let's generate predictions for about 1/3 of our data using our baseline classifiers and compare their accuracy:

In [129]:
test = generate_random_instances(titanic_data, 700)

predictions_random = random_classifier(titanic_data, 700, 3)
predictions_zero = zero_r_classifier(titanic_data, 700, 3)

print("Random Classifier Accuracy:", compute_accuracy(predictions_random, [x[3] for x in test]))
print("Zero Rule Classifier Accuracy:", compute_accuracy(predictions_zero, [x[3] for x in test]))

Random Classifier Accuracy: 0.5557142857142857
Zero Rule Classifier Accuracy: 0.6657142857142857


As we can see, these incredibly simple baseline classifiers actually do fairly well for the data, with accuracy between 50-70%. However, our actual classifiers do seem to consistantly be more accurate, although the increase in complexity is much greater than the increase in accuracy for both Naive Bayes and k-NN compared to these simple predictors.